In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from run_glue import set_seed, load_and_cache_examples, train, evaluate 

from transformers import (CamembertConfig,
                          CamembertForSequenceClassification,
                          CamembertTokenizer,
                         DistilBertTokenizer)
from torch import device


from transformers import AdamW, get_linear_schedule_with_warmup

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

from namespace import Namespace, as_namespace

In [3]:
path_data = "data/"

MODEL_CLASSES = {
    'camembert': (CamembertConfig, CamembertForSequenceClassification, CamembertTokenizer),
}


In [4]:
args = Namespace({"adam_epsilon": 1e-08,
                 "cache_dir": '',
                 "config_name": '',
                 "data_dir": path_data,
                 "device": device("cpu"),
                  "do_eval": False,
                  "do_lower_case": True,
                  "do_train": True,
                  "eval_all_checkpoints": False,
                  "evaluate_during_training": False,
                  "fp16": False,
                  "fp16_opt_level": 'O1',
                  "gradient_accumulation_steps": 1,
                  "learning_rate": 2e-05,
                  "local_rank": -1,
                  "logging_steps": 50,
                  "max_grad_norm": 1.0,
                  "max_seq_length": 128,
                  "max_steps": -1,
                  "model_name_or_path": "camembert-base",#'bert-base-cased',
                  "model_type": 'camembert',
                  "n_gpu": 0,
                  "no_cuda": True,
                  "num_train_epochs": 1, #must be int in fact !
                  "output_dir":'/tmp/result',
                  "output_mode":'classification',
                  "overwrite_cache":True,
                  "overwrite_output_dir":True,
                  "per_gpu_eval_batch_size":8,
                  "per_gpu_train_batch_size":32,
                  "save_steps":50,
                  "seed":42,
                  "server_ip":'',
                  "server_port":'',
                  "task_name":'anna',
                  "tokenizer_name":'',
                  "tpu": False,
                  "tpu_ip_address": '',
                  "tpu_name": '',
                  "warmup_steps": 0,
                  "weight_decay": 0.0,
                  "xrt_tpu_config": ''}
                )

NameError: name 'Namespace' is not defined

In [27]:
"knowledge_20191112.tsv"

'knowledge_20191112.tsv'

In [28]:
# Set seed
set_seed(args)

# Prepare GLUE task
args.task_name = args.task_name.lower()
if args.task_name not in processors:
    raise ValueError("Task not found: %s" % (args.task_name))
processor = processors[args.task_name]()
args.output_mode = output_modes[args.task_name]
label_list = processor.get_labels()
num_labels = len(label_list)

In [29]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]

In [30]:
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                      num_labels=num_labels,
                                      finetuning_task=args.task_name,
                                      cache_dir=args.cache_dir if args.cache_dir else None)

In [31]:
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
                                            do_lower_case=args.do_lower_case,
                                            cache_dir=args.cache_dir if args.cache_dir else None)

In [32]:
model = model_class.from_pretrained(args.model_name_or_path,
                                    from_tf=bool('.ckpt' in args.model_name_or_path),
                                    config=config,
                                    cache_dir=args.cache_dir if args.cache_dir else None)

In [35]:
train_dataset = load_and_cache_examples(args=args,
                                        task="anna",
                                        tokenizer=tokenizer,
                                        evaluate=False)

anna
<class 'transformers.data.processors.glue.AnnaProcessor'>


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
len(train_dataset)

7362

In [37]:
train_dataset.tensors

(tensor([[    5, 28306,    21,  ...,     1,     1,     1],
         [    5,   114,   484,  ...,     1,     1,     1],
         [    5,  7363,    56,  ...,     1,     1,     1],
         ...,
         [    5,  1470,   279,  ...,     1,     1,     1],
         [    5,  2302,    30,  ...,     1,     1,     1],
         [    5, 30012,    35,  ...,     1,     1,     1]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 tensor([945, 141, 330,  ..., 406, 201, 747]))

In [38]:
train(args=args,
      train_dataset=train_dataset,
      model=model,
      tokenizer=tokenizer)



Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


Iteration:   0%|          | 0/231 [00:00<?, ?it/s]

{'input_ids': tensor([[   5,  841, 8241,  ...,    1,    1,    1],
        [   5,  100, 1282,  ...,    1,    1,    1],
        [   5, 2302,   30,  ...,    1,    1,    1],
        ...,
        [   5, 4171,    7,  ...,    1,    1,    1],
        [   5,  841,   85,  ...,    1,    1,    1],
        [   5, 1470,  279,  ...,    1,    1,    1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([596, 123, 574, 416, 770, 146, 599, 870, 732, 604,  81, 741, 737, 600,
        900,  91, 214, 604, 221, 228,  87, 510, 342, 350, 786, 516, 214, 597,
        443, 562, 724, 406]), 'token_type_ids': None}
torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32])
tensor(900)
tensor(81)
tensor([ 81,  87,  91, 123, 146, 214, 221, 228, 342, 350, 406, 416, 443, 510,
        516, 562, 574, 596, 597, 599, 60




Iteration:   0%|          | 1/231 [00:41<2:39:12, 41.53s/it]

{'input_ids': tensor([[    5,   175,    26,  ...,     1,     1,     1],
        [    5,  7363,    56,  ...,     1,     1,     1],
        [    5, 14050,    20,  ...,     1,     1,     1],
        ...,
        [    5,  3053,    30,  ...,     1,     1,     1],
        [    5,   841,    48,  ...,     1,     1,     1],
        [    5, 15294,  4171,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([ 189,  771,  736,  421,  732,  712,  201,   76,  181,  726,  762,  333,
         345,  128,  745,  907,  222,  923,  318,  421,  521,    0,  405,  234,
        1042,  704,  184,  715,  469,  565,  779,  247]), 'token_type_ids': None}
torch.Size([32, 128])
torch.Size([32, 128])
torch.Size([32])
tensor(1042)
tensor(0)
tensor([   0,   76,  128,  181,  184,  189,  201, 

KeyboardInterrupt: 